In [1]:
# importation des modèles
from tensorflow.keras.models import load_model
model1 = load_model("./models/detection_model.h5")
model2 = load_model("./models/classification_model.h5")

c:\users\mizou\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mizou\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mizou\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mizou\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [2]:
# création de la liste des panneaux qui nous servent dans l'affichage de résultat de classification
signs = ["interdit de depasser","vitesse max 70","obligation de tourner a gauche","danger","stop"]

In [3]:
def decision1(model,objct): # prediction modèle ou non
    prediction = model.predict(objct)
    l = prediction.tolist()
    return(l[0][0])

In [4]:
def decision2(img,model,objct):  # prediction de la classe
    prediction = model.predict(objct)
    l = prediction.tolist()
    X = l[0].index(max(l[0]))  # l'indice du panneau qui a la plus grande probabilité
    img = cv2.putText(img,signs[X],(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2,cv2.LINE_AA) #Affichage

In [8]:
def decision_final(img,objct):
    objct = cv2.resize(objct, dsize=(40,40), interpolation=cv2.INTER_CUBIC)
    objct = cv2.cvtColor(objct, cv2.COLOR_BGR2GRAY)
    objct = objct / 255
    objct = np.reshape(objct,(1,40,40,1))
    if decision1(model1,objct)>0.3:
        cv2.rectangle(img, (y1, x1), (y2, x2), (255, 0, 0), 2) # Cadre autour du panneau
        decision2(img,model2,objct)

In [ ]:
import cv2 
import numpy as np
cap = cv2.VideoCapture(0)
while(1):
    _, frame = cap.read()   # capture d'un frame
    
    Height, Width = frame.shape[:2]    # caractéristiques de la fenêtre
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)    #Conversion en HSV de la capture
    lower_red1 = np.array([0,100,100])             # la couleur rouge en hsv (première partie)
    upper_red1 = np.array([10,255,255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    lower_red2 = np.array([160,100,100])            # la couleur rouge en hsv (deuxième partie)
    upper_red2 = np.array([179,255,255])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)  
    lower_blue = np.array([100,150,0])               ## la couleur blue en hsv
    upper_blue = np.array([140,255,255])
    mask3 = cv2.inRange(hsv, lower_blue, upper_blue)
    mask = mask1 + mask2 + mask3      # construction de masque final
    
    contours, _ = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # trouver les contours d'un objet détecté
    
    if len(contours) > 0:
    # Obtenir le plus grand contour circulaire de l'objet 
        c = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(c)  
        M = cv2.moments(c)   
        try:
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))  #Centre de masse : moments centrées 

        
        except:
            center =   int(Height/2), int(Width/2)
        
        try:
            if int(radius) > 50:
                x1,y1 = center[1]-int(radius)-10,center[0]-int(radius)-10
                d = int(radius)*2
                x2 , y2 = x1 + d + 20 , y1 + d +20
                objct = frame[x1:x2,y1:y2,:]
                decision_final(frame,objct)
        except:
            continue
    
    text = "Cliquer sur 'ECHAP' pour quitter ... "
    img1 = cv2.putText(frame,text,(5,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1,cv2.LINE_AA)
        
        
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) == 27 :  # echap pour quitter
        break
cap.release()
cv2.destroyAllWindows()
